In [1]:
# Translated to .py by Evanthia Kazagli
# 2017
# Adapted for PandasBiogeme by Michel Bierlaire
# Thu Nov  1 18:02:14 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.models import lognested, logcnl_avail
from biogeme.expressions import Beta, DefineVariable

df = pd.read_csv("swissmetro.dat",'\t')
database = db.Database("swissmetro",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)


exclude = ((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  ) + (  CHOICE   ==  0  ) + ( AGE == 6 ))>0
database.remove(exclude)


#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR	 = Beta('ASC_CAR',0,None,None,0)
ASC_SBB	 = Beta('ASC_SBB',0,None,None,1)
ASC_SM	 = Beta('ASC_SM',0,None,None,0)
B_HE	 = Beta('B_HE',0,None,None,0)
B_COST	 = Beta('B_COST',0,None,None,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,None,None,0)
B_SBB_TIME	 = Beta('B_SBB_TIME',0,None,None,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,None,None,0)
B_SENIOR	 = Beta('B_SENIOR',0,None,None,0)
B_GA	 = Beta('B_GA',0,None,None,0)

# Nest parameters
Rail_based = Beta('Rail_based',1,1,None,0)
classic = Beta('classic',1,1,None,0)

# Alphsa parameters
Rail_based_SM = Beta('Rail_based_SM',1,1e-05,1,1)
Rail_based_Train = Beta('Rail_based_Train',0.5,1e-05,1,0)
classic_Car = Beta('classic_Car',1,1e-05,1,1)
classic_Train = 1 - Rail_based_Train #

# Define here arithmetic expressions for name that are not directly 
# available from the data
SENIOR  = DefineVariable('SENIOR', AGE   ==  5 ,database)
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)
TRAIN_HE_SCALED = DefineVariable('TRAIN_HE_SCALED', TRAIN_HE / 100,database)
SM_HE_SCALED = DefineVariable('SM_HE_SCALED', SM_HE / 100,database)

# Utilities
Car_SP = ASC_CAR + B_CAR_TIME * CAR_TT_SCALED + B_COST * CAR_CO_SCALED + B_SENIOR * SENIOR
SBB_SP = ASC_SBB + B_SBB_TIME * TRAIN_TT_SCALED + B_COST * TRAIN_COST_SCALED + B_HE * TRAIN_HE_SCALED + B_GA * GA
SM_SP = ASC_SM + B_SM_TIME * SM_TT_SCALED + B_COST * SM_COST_SCALED + B_HE * SM_HE_SCALED + B_GA * GA + B_SENIOR * SENIOR
V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}


# Definition of nests
alpha_Rail_based = {1: Rail_based_Train, 2: Rail_based_SM, 3: 0}
alpha_classic = {1: classic_Train, 2: 0, 3: classic_Car}

nest_Rail_based = Rail_based, alpha_Rail_based
nest_classic = classic, alpha_classic

nests = nest_Rail_based, nest_classic

# CNL (Cross-Nested Logit Model), with availability conditions
logprob = logcnl_avail(V, av, nests, CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MEV_SM_CNL_var"
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

# Compare with nested logit
print("*** Compare with nested logit ***")
# Definition of nests
MU_classic = Beta('MU_classic',1,1,None,0)
nl_innovative = 1.0, [2]
nl_classic = MU_classic, [1, 3]
nl_nests = nl_classic, nl_innovative
nl_logprob = lognested(V,av,nl_nests,CHOICE)
biogeme_nl  = bio.BIOGEME(database,nl_logprob)
biogeme_nl.modelName = "MEV_SM_CNL_var_NL"
results_nl = biogeme_nl.estimate()

ll_nl = results_nl.data.logLike
rhobar_nl = results_nl.data.rhoBarSquare
ll_nested = results.data.logLike
rhobar_nested = results.data.rhoBarSquare

print(f"LL nested:  {ll_nl:.3f}  rhobar: {rhobar_nl:.3f}  Parameters: {results_nl.data.nparam}")
print(f"LL cross-nested: {ll_nested:.3f}  rhobar: {rhobar_nested:.3f}  Parameters: {results.data.nparam}")
lr = -2 * (ll_nl - ll_nested)
print(f"Likelihood ratio: {lr:.3f}")


print("*** Compare with cross-nested logit with fixed alphas ***")
# Definition of nests
cnlfix_alpha_Rail_based = {1: 0.5, 2: 1, 3: 0}
cnlfix_alpha_classic = {1: 0.5, 2: 0, 3: 1}

cnlfix_Rail_based = Rail_based, cnlfix_alpha_Rail_based
cnlfix_classic = classic, cnlfix_alpha_classic

cnlfix_nests = cnlfix_Rail_based, cnlfix_classic

cnlfix_logprob = logcnl_avail(V,av,cnlfix_nests,CHOICE)
biogeme_cnlfix  = bio.BIOGEME(database,cnlfix_logprob)
biogeme_cnlfix.modelName = "MEV_SM_CNL_var_fix"
results_cnlfix = biogeme_cnlfix.estimate()

ll_cnlfix = results_cnlfix.data.logLike
rhobar_cnlfix = results_cnlfix.data.rhoBarSquare
ll_nested = results.data.logLike
rhobar_nested = results.data.rhoBarSquare

print(f"LL nested:  {ll_cnlfix:.3f}  rhobar: {rhobar_cnlfix:.3f}  Parameters: {results_cnlfix.data.nparam}")
print(f"LL cross-nested: {ll_nested:.3f}  rhobar: {rhobar_nested:.3f}  Parameters: {results.data.nparam}")
lr = -2 * (ll_cnlfix - ll_nested)
print(f"Likelihood ratio: {lr:.3f}")


C:\anaconda3\envs\study\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


                  Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_CAR          -0.651   0.0847   -7.68 1.55e-14         0.105        -6.17   
ASC_SM           -0.267   0.0712   -3.75 0.000175        0.0808        -3.31   
B_CAR_TIME       -0.706   0.0607   -11.6        0         0.125        -5.64   
B_COST           -0.755   0.0456   -16.6        0        0.0603        -12.5   
B_GA              0.739    0.102    7.26 3.89e-13         0.103         7.15   
B_HE             -0.331   0.0561    -5.9 3.61e-09        0.0583        -5.68   
B_SBB_TIME       -0.887   0.0679   -13.1        0         0.114         -7.8   
B_SENIOR         -0.839    0.084   -9.99        0           0.1        -8.37   
B_SM_TIME        -0.776   0.0754   -10.3        0          0.15        -5.17   
Rail_based         3.73    0.488    7.65 2.04e-14         0.489         7.63   
Rail_based_Train  0.527   0.0301    17.5        0        0.0347         15.2   
classic            2.41    0.172      14